<a href="https://colab.research.google.com/github/Bathalaxxx/Training-Codes-Repository-EJGP-/blob/main/Ultimate_Options_Trading_(4H_TF)_Version_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from google.colab import files

def upload_csv():
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    return pd.read_csv(filename)

def display_data(data):
    print("Uploaded CSV Data:")
    print(data.head())

def split_data(data):
    X = data.drop('Predicted Price Up/down by 3%', axis=1)
    y = data['Predicted Price Up/down by 3%']
    return train_test_split(X, y, test_size=0.3, random_state=42)

def scale_features(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    feature_names = X_train.columns
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_names)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_names)
    return X_train_scaled, X_test_scaled, scaler, feature_names

def train_evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=1.0)
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    return {'Model': model, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1-Score': f1}

def print_model_performances(model_performances):
    df_performance = pd.DataFrame(model_performances)
    best_model = df_performance.sort_values('Accuracy', ascending=False).iloc[0]

    print('\nModel Performances:')
    print(df_performance)
    print('\nBest Model:', best_model['Model'])

    return best_model['Model']

def make_predictions(chosen_model, scaler, feature_names):
    make_prediction = True
    while make_prediction:
        print('\nEnter predictor values for making predictions:')
        new_predictors = []
        for feature in feature_names:
            value = input(f'{feature}: ')
            new_predictors.append(float(value))

        new_predictors_scaled = scaler.transform([new_predictors])
        prediction = chosen_model.predict(new_predictors_scaled)

        print('\nPrediction:')
        print('Predicted Price Up/down by 3%:', prediction[0])

        choice = input('\nDo you want to make another prediction? (yes/no): ')
        make_prediction = choice.lower() == 'yes'

# Upload CSV file
data = upload_csv()

# Display the uploaded CSV data
display_data(data)

# Handling missing data
data.dropna(inplace=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = split_data(data)

# Feature scaling
X_train_scaled, X_test_scaled, scaler, feature_names = scale_features(X_train, X_test)

# Define the models to compare
models = [
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('Random Forest', RandomForestClassifier()),
    ('Gradient Boosting', GradientBoostingClassifier()),
    ('AdaBoost', AdaBoostClassifier()),
    ('Support Vector Machine', SVC())
]

# Hyperparameter tuning
param_grid = {
    'Logistic Regression': {'C': [0.1, 1, 10]},
    'Decision Tree': {'max_depth': [None, 5, 10]},
    'Random Forest': {'n_estimators': [50, 100, 200]},
    'Gradient Boosting': {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.01]},
    'AdaBoost': {'n_estimators': [50, 100, 200], 'learning_rate': [0.1, 0.01]},
    'Support Vector Machine': {'C': [0.1, 1, 10]}
}

model_performances = []
for name, model in models:
    pipeline = GridSearchCV(model, param_grid[name], cv=5)
    performance = {'Model': name}
    performance.update(train_evaluate_model(pipeline, X_train_scaled, y_train, X_test_scaled, y_test))
    model_performances.append(performance)

# Print model performances and select the best model
chosen_model = print_model_performances(model_performances)

# Additional steps for further analysis and decision-making can be added here

# Make predictions using the chosen model
make_predictions(chosen_model, scaler, feature_names)


Saving Options Trading Data.csv to Options Trading Data (1).csv
Uploaded CSV Data:
   Price Range (H-L)    Close  Monday  Tuesday  Wednesday  Thursday  Friday  \
0              88.67  4349.99       0        0          0         1       0   
1             152.07  4427.30       0        0          0         1       0   
2             151.97  4352.34       0        0          0         1       0   
3             154.10  4325.23       0        0          0         0       1   
4             111.13  4285.08       0        0          0         0       1   

   Saturday  Sunday  Predicted Price Up/down by 3%  
0         0       0                              0  
1         0       0                              0  
2         0       0                              0  
3         0       0                             -1  
4         0       0                             -1  

Model Performances:
                                               Model  Accuracy  Precision  \
0  GridSearchCV(cv=5, esti

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(



Prediction:
Predicted Price Up/down by 3%: 0

Do you want to make another prediction? (yes/no): no
